# Reducing Data From Feder Observatory 

This document describes the choices you need to make and the settings you need to use to implement those choices.

# Camera: Apogee Alta U9

This is the default camera on the telescope. Some basic properties of the camera are:

+ The imaging region is up to 3073×2048 pixels (can be smaller if you bin the images).
+ An optional overscan region extends the image size to 3085×2048 pixels. 
+ The part of the overscan that is useful for estimating bias is from column 3076 to 3079. The rest of the columns get some light leak if the surrounding pixels are at high values (e.g. when taking a flat).
+ Images are 16-bit, so the maximum pixel value is 65,535.
+ Pixels begin to deviate from linearity around 50,000 counts.
+ *Typical* dark current is 0.1 $e^{-}$/pix/sec, but roughly 0.1% of pixels are significantly hotter.
+ Read noise is roughly 30 $e^{-}$/pix.
+ The camera gain is 1.5.

## Overall data reduction strategy and advice

+ Whenever possible, use darks that are the same exposure time as the flat/light images and do not subtract bias.
+ If there are not darks with the right exposure times, try to scale the dark frames down (e.g. from 30 to 15 second) rather than up (e.g. from 30 to 60 seconds).
+ Sometimes you will have to scale darks up because you have no choice.
+ Clip outliers to remove cosmic rays in the calibration images.
+ Trim off the overscan.
+ Use flats; if there are no flats on the night you are reducing, try to find a night close to the time of your night that does have flats and use those.
+ Some times there will be no appropriate flats. 


## Specific settings

### Reduction steps

+ **Overscan**: Do not subtract it unless you know you need to. If you subtract it, then you **must subtract overscan from all images** and you want these settings:
    + Overscan region is all of `axis 0` and from `3076` to `3079` along the other axis.
    + Fit a polynomial (0th order, or maybe first order).
+ **Trim:** Trim the images if they have overscan. *If you trim one set of images you must trim them all.* The settings are:
    + Keep all of `axis 0` and from `0` to `3073` along the other axis.
+ **Subtract bias:** Do this only if you absolutely must (because you need to scale the darks). 
+ **Subtract dark:** Always. 
+ **Flat correct:** Do for light images....unless there are no flats.

### Image combining

+ **Clip before combining:** For *calibration images* you should do this. Using sigma clip with both low and high threshold to 5 works reasonably well. 
+ **Combine images:** Typically you shuold choose to combine using average. If your goal in choosing median is to reject cosmic rays (for example), then clipping before combining works better.
    + Only **scale** before combining when combining flats. This scales all images to so that their mdeian or average value is 1, then combines them. When combining flats this is almost always the right thing to do. 
+ **Group by:** This setting controls how the calibrated images are grouped to make masters. For bias and light images you usually don't want to set this. For darks it makes sense to combine only the dark images that have the same exposure time. 
    + By default, reducer sets "Group by" to `exposure` for darks; depending onyour setup it might (or might not) make sense to also group by temperature.
    + By default, reducer set "Group by" to `exposure, filter` for flats, since flats with different filters should never be combined. That setting works fine for dome flats, but for sky flats, in which the exposure time is typically different for each image, the proper setting is just `filter`.
